In [1]:
import torch
import numpy as np
import random
random.seed(1)
torch.cuda.manual_seed(1)
torch.manual_seed(1)
np.random.seed(1)
from NeuralPITF import AttentionPITF, SinglePITF_Loss, DataSet, RNNAttentionPITF, TagAttentionPITF
from torch.autograd import Variable
# from torch.utils import data
import torch.optim as optim
import datetime

In [2]:
train_data_path = 'data/movielens/all_id_core3_train'
test_data_path = 'data/movielens/all_id_core3_test'
# ini_time = 1135429431000

# train_data_path = 'data/movielens/all_id_core1_train'
# test_data_path = 'data/movielens/all_id_core1_test'

# train_data_path = 'data/lastFM/all_id_core1_train'
# test_data_path = 'data/lastFM/all_id_core1_test'

# train_data_path = 'data/lastFM/all_id_core3_train'
# test_data_path = 'data/lastFM/all_id_core3_test'

# train_data_path = 'data/movielens/all_id_core3_train'
# test_data_path = 'data/movielens/all_id_core3_test'

# train_data_path = 'data/delicious/all_id_core3_train'
# test_data_path = 'data/delicious/all_id_core3_test'

movielens_all = np.genfromtxt(train_data_path, delimiter='\t', dtype=float)
ini_time = int(movielens_all[:, 3].min())
movielens_all[:, -1] = (movielens_all[:, -1] - ini_time) / (24*3600*1000)
movielens = movielens_all.astype(int)

movielens_test_all = np.genfromtxt(test_data_path, delimiter='\t', dtype=float)
movielens_test_all[:, -1] = (movielens_test_all[:, -1] - ini_time) / (24*3600*1000)
movielens_test = movielens_test_all.astype(int)

user_vecs_path = 'PreVecs/movielens/UserVecs.txt'
item_vecs_path = 'PreVecs/movielens/ItemVecs.txt'
tag_user_vec_path = 'PreVecs/movielens/UserTagVecs.txt'
tag_item_vec_path = 'PreVecs/movielens/ItemTagVecs.txt'


def handle_pre_vecs(file_path):
    pre_vecs = np.genfromtxt(file_path, delimiter='\t', dtype=str)
    a = pre_vecs[:, 0]
    b = pre_vecs[:, -1]
    for i in range(len(a)):
        a[i] = a[i].replace('[', '')        
    for i in range(len(b)):
        b[i] = b[i].replace(']', '')   
    pre_vecs[:, 0] = a
    pre_vecs[:, -1] = b  
    pre_vecs = pre_vecs.astype(float)
    return torch.FloatTensor(pre_vecs)


user_vecs = handle_pre_vecs(user_vecs_path)
item_vecs = handle_pre_vecs(item_vecs_path)
tag_user_vecs = handle_pre_vecs(tag_user_vec_path)
tag_item_vecs = handle_pre_vecs(tag_item_vec_path)
ini_embeddings = [user_vecs, item_vecs, tag_user_vecs, tag_item_vecs]

In [12]:
import NeuralPITF
from imp import reload
reload(NeuralPITF)
from NeuralPITF import AttentionPITF, SinglePITF_Loss, DataSet, RNNAttentionPITF, TagAttentionPITF
def train(data, test, m, gamma):
    """
    该函数主要作用： 定义网络；定义数据，定义损失函数和优化器，计算重要指标，开始训练（训练网络，计算在测试集上的指标）
    主要需要调整的参数： m 与 gamma
    
    :return:
    """
    learnRate = 0.001
    lam = 0.00005
    dim = 64
    iter_ = 100
    init_st = 0.01
    m = m
    gamma = gamma
    batch_size = 100
    n = 1000
    # 计算numUser, numItem, numTag
    dataload = DataSet(data, test, True)
    num_user, num_item, num_tag = dataload.calc_number_of_dimensions()
    model = RNNAttentionPITF(int(num_user), int(num_item), int(num_tag), dim, init_st, m, gamma, ini_embeddings).cuda()
    # model = AttentionPITF(int(num_user), int(num_item), int(num_tag), dim, init_st, m, gamma, ini_embeddings).cuda()
    # model = TagAttentionPITF(int(num_user), int(num_item), int(num_tag), dim, init_st, m, gamma).cuda()
    # torch.save(model.state_dict(), 'attention_initial_params')
    # 对每个正样本进行负采样
    loss_function = SinglePITF_Loss().cuda()
    opti = optim.SGD(model.parameters(), lr=learnRate, weight_decay=lam)
    # opti = optim.Adam(model.parameters(), lr=learnRate, weight_decay=lam)
    opti.zero_grad()    
    # 每个epoch中的sample包含一个正样本和j个负样本
    best_result = 0
    # best_result_state = model.state_dict()
    # best_file = open('Attention_best_params.txt', 'a')
    for epoch in range(iter_):
        # file_ = open('AttentionTureParam.txt', 'a')
        all_data = dataload.get_sequential(num_tag, m, 100)
        all_data = all_data[:, :4 + m]
        losses = []
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        for i, batch in enumerate(dataload.get_batch(all_data, batch_size)):
            # print(batch)
            # input_ = dataload.draw_negative_sample(num_tag, sample, True)
            r = model(torch.LongTensor(batch).cuda())
            opti.zero_grad()
            # print(model.embedding.userVecs.weight)
            loss = loss_function(r)
            # print(loss)
            loss.backward()
            opti.step()
            losses.append(loss.data)
            if i % n == 0:
                print ("[%02d/%d] [%03d/%d] mean_loss : %0.2f" %(epoch, iter_, i, len(all_data)/batch_size, np.mean(losses)))
                losses = []
        precision = 0
        recall = 0
        count = 0
        validaTagSet = dataload.validaTagSet

        for u in validaTagSet.keys():
            for i in validaTagSet[u].keys():
                number = 0
                tags = validaTagSet[u][i]
                tagsNum = len(tags)
                x_t = torch.LongTensor([u, i] + list(dataload.userShortMemory[u][:m])).cuda()
                x_t = x_t.unsqueeze(0)
                y_pre = model.predict_top_k(x_t)
                # print(y_pre)
                for tag in y_pre[0]:
                    if int(tag) in tags:
                        number += 1
                precision = precision + float(number / 5)
                recall = recall + float(number / tagsNum)
                count += 1
        precision = precision / count
        recall = recall / count
        if precision==0 and recall == 0:
            f_score = 0
        else:
            f_score = 2 * (precision * recall) / (precision + recall)
        print("Precisions: " + str(precision))
        print("Recall: " + str(recall))
        print("F1: " + str(f_score))
        # 将模型最好时的效果保存下来
        if f_score > best_result:
            best_result = f_score
            # best_result_state = model.state_dict()
        print("best result: " + str(best_result))
        print("==================================")
        # info = " [%02d/%d] gamma: %f the length m: %d " %(epoch, iter_, gamma, m)   
        # file_.write(info + '\n')
        # file_.write("Precision: " + str(precision) + "  Recall: " + str(recall)+ " F1: " + str(f_score) + " Best Result: " + str(best_result))
        # file_.write('\r\n')
    # torch.save(model, "net.pkl")
    # torch.save(best_result_state, "attention_net_params.pkl")
    # best_file.write('gamma: %f,  the length: %d, best_result: %f ' %(gamma, m, best_result)+'\r\n')
    # best_file.close()

m_params = [8]
gamma_params = [0.5]
# m_params = [1,2,4,5,6,8,10]
# gamma_params = [0.2,0.4,0.5, 0.6,0.8,1]
for m in m_params:
    for gamma in gamma_params:
        train(movielens, movielens_test, m, gamma)

/home/cbd109/applications/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Use the BPR for Optimization
2018-10-11 09:07:15
[00/100] [000/29931] mean_loss : 8.65
[00/100] [1000/29931] mean_loss : 0.85
[00/100] [2000/29931] mean_loss : 0.86
[00/100] [3000/29931] mean_loss : 0.52
[00/100] [4000/29931] mean_loss : 0.38
[00/100] [5000/29931] mean_loss : 0.39
[00/100] [6000/29931] mean_loss : 0.33
[00/100] [7000/29931] mean_loss : 0.24
[00/100] [8000/29931] mean_loss : 0.25
[00/100] [9000/29931] mean_loss : 0.19
[00/100] [10000/29931] mean_loss : 0.16
[00/100] [11000/29931] mean_loss : 0.14
[00/100] [12000/29931] mean_loss : 0.14
[00/100] [13000/29931] mean_loss : 0.12
[00/100] [14000/29931] mean_loss : 0.10
[00/100] [15000/29931] mean_loss : 0.12
[00/100] [16000/29931] mean_loss : 0.12
[00/100] [17000/29931] mean_loss : 0.12
[00/100] [18000/29931] mean_loss : 0.13
[00/100] [19000/29931] mean_loss : 0.14
[00/100] [20000/29931] mean_loss : 0.14
[00/100] [21000/29931] mean_loss : 0.13
[00/100] [22000/29931] mean_loss : 0.11
[00/100] [23000/29931] mean_loss : 0.11
[0

[06/100] [000/29931] mean_loss : 0.02
[06/100] [1000/29931] mean_loss : 0.03
[06/100] [2000/29931] mean_loss : 0.04
[06/100] [3000/29931] mean_loss : 0.03
[06/100] [4000/29931] mean_loss : 0.04
[06/100] [5000/29931] mean_loss : 0.04
[06/100] [6000/29931] mean_loss : 0.04
[06/100] [7000/29931] mean_loss : 0.04
[06/100] [8000/29931] mean_loss : 0.03
[06/100] [9000/29931] mean_loss : 0.04
[06/100] [10000/29931] mean_loss : 0.04
[06/100] [11000/29931] mean_loss : 0.04
[06/100] [12000/29931] mean_loss : 0.04
[06/100] [13000/29931] mean_loss : 0.04
[06/100] [14000/29931] mean_loss : 0.04
[06/100] [15000/29931] mean_loss : 0.04
[06/100] [16000/29931] mean_loss : 0.04
[06/100] [17000/29931] mean_loss : 0.04
[06/100] [18000/29931] mean_loss : 0.04
[06/100] [19000/29931] mean_loss : 0.04
[06/100] [20000/29931] mean_loss : 0.04
[06/100] [21000/29931] mean_loss : 0.04
[06/100] [22000/29931] mean_loss : 0.04
[06/100] [23000/29931] mean_loss : 0.04
[06/100] [24000/29931] mean_loss : 0.04
[06/100] [2

[12/100] [2000/29931] mean_loss : 0.03
[12/100] [3000/29931] mean_loss : 0.03
[12/100] [4000/29931] mean_loss : 0.03
[12/100] [5000/29931] mean_loss : 0.03
[12/100] [6000/29931] mean_loss : 0.03
[12/100] [7000/29931] mean_loss : 0.03
[12/100] [8000/29931] mean_loss : 0.03
[12/100] [9000/29931] mean_loss : 0.04
[12/100] [10000/29931] mean_loss : 0.04
[12/100] [11000/29931] mean_loss : 0.04
[12/100] [12000/29931] mean_loss : 0.04
[12/100] [13000/29931] mean_loss : 0.04
[12/100] [14000/29931] mean_loss : 0.04
[12/100] [15000/29931] mean_loss : 0.04
[12/100] [16000/29931] mean_loss : 0.04
[12/100] [17000/29931] mean_loss : 0.03
[12/100] [18000/29931] mean_loss : 0.04
[12/100] [19000/29931] mean_loss : 0.04
[12/100] [20000/29931] mean_loss : 0.04
[12/100] [21000/29931] mean_loss : 0.04
[12/100] [22000/29931] mean_loss : 0.04
[12/100] [23000/29931] mean_loss : 0.04
[12/100] [24000/29931] mean_loss : 0.04
[12/100] [25000/29931] mean_loss : 0.04
[12/100] [26000/29931] mean_loss : 0.04
[12/100]

[18/100] [4000/29931] mean_loss : 0.03
[18/100] [5000/29931] mean_loss : 0.03
[18/100] [6000/29931] mean_loss : 0.03
[18/100] [7000/29931] mean_loss : 0.03
[18/100] [8000/29931] mean_loss : 0.03
[18/100] [9000/29931] mean_loss : 0.04
[18/100] [10000/29931] mean_loss : 0.04
[18/100] [11000/29931] mean_loss : 0.03
[18/100] [12000/29931] mean_loss : 0.04
[18/100] [13000/29931] mean_loss : 0.04
[18/100] [14000/29931] mean_loss : 0.04
[18/100] [15000/29931] mean_loss : 0.04
[18/100] [16000/29931] mean_loss : 0.03
[18/100] [17000/29931] mean_loss : 0.04
[18/100] [18000/29931] mean_loss : 0.04
[18/100] [19000/29931] mean_loss : 0.03
[18/100] [20000/29931] mean_loss : 0.04
[18/100] [21000/29931] mean_loss : 0.03
[18/100] [22000/29931] mean_loss : 0.03
[18/100] [23000/29931] mean_loss : 0.05
[18/100] [24000/29931] mean_loss : 0.04
[18/100] [25000/29931] mean_loss : 0.04
[18/100] [26000/29931] mean_loss : 0.04
[18/100] [27000/29931] mean_loss : 0.04
[18/100] [28000/29931] mean_loss : 0.05
[18/10

...

[72/100] [20000/29931] mean_loss : 0.04
[72/100] [21000/29931] mean_loss : 0.04
[72/100] [22000/29931] mean_loss : 0.04
[72/100] [23000/29931] mean_loss : 0.05
[72/100] [24000/29931] mean_loss : 0.04
[72/100] [25000/29931] mean_loss : 0.05
[72/100] [26000/29931] mean_loss : 0.05
[72/100] [27000/29931] mean_loss : 0.05
[72/100] [28000/29931] mean_loss : 0.05
[72/100] [29000/29931] mean_loss : 0.05
Precisions: 0.12987804878048864
Recall: 0.515962543554007
F1: 0.20751934516344292
best result: 0.2166719467907358
2018-10-11 11:44:02
[73/100] [000/29931] mean_loss : 0.00
[73/100] [1000/29931] mean_loss : 0.02
[73/100] [2000/29931] mean_loss : 0.02
[73/100] [3000/29931] mean_loss : 0.03
[73/100] [4000/29931] mean_loss : 0.03
[73/100] [5000/29931] mean_loss : 0.03
[73/100] [6000/29931] mean_loss : 0.03
[73/100] [7000/29931] mean_loss : 0.03
[73/100] [8000/29931] mean_loss : 0.03
[73/100] [9000/29931] mean_loss : 0.03
[73/100] [10000/29931] mean_loss : 0.04
[73/100] [11000/29931] mean_loss : 0.

[78/100] [22000/29931] mean_loss : 0.05
[78/100] [23000/29931] mean_loss : 0.05
[78/100] [24000/29931] mean_loss : 0.05
[78/100] [25000/29931] mean_loss : 0.05
[78/100] [26000/29931] mean_loss : 0.05
[78/100] [27000/29931] mean_loss : 0.05
[78/100] [28000/29931] mean_loss : 0.05
[78/100] [29000/29931] mean_loss : 0.05
Precisions: 0.13079268292683013
Recall: 0.5216790069686411
F1: 0.20914868186530547
best result: 0.2166719467907358
2018-10-11 11:56:19
[79/100] [000/29931] mean_loss : 0.00
[79/100] [1000/29931] mean_loss : 0.02
[79/100] [2000/29931] mean_loss : 0.02
[79/100] [3000/29931] mean_loss : 0.03
[79/100] [4000/29931] mean_loss : 0.03
[79/100] [5000/29931] mean_loss : 0.03
[79/100] [6000/29931] mean_loss : 0.03
[79/100] [7000/29931] mean_loss : 0.03
[79/100] [8000/29931] mean_loss : 0.03
[79/100] [9000/29931] mean_loss : 0.03
[79/100] [10000/29931] mean_loss : 0.04
[79/100] [11000/29931] mean_loss : 0.05
[79/100] [12000/29931] mean_loss : 0.05
[79/100] [13000/29931] mean_loss : 0

[84/100] [24000/29931] mean_loss : 0.05
[84/100] [25000/29931] mean_loss : 0.05
[84/100] [26000/29931] mean_loss : 0.05
[84/100] [27000/29931] mean_loss : 0.05
[84/100] [28000/29931] mean_loss : 0.05
[84/100] [29000/29931] mean_loss : 0.05
Precisions: 0.1283536585365862
Recall: 0.5121515679442509
F1: 0.20526460910264044
best result: 0.2166719467907358
2018-10-11 12:08:45
[85/100] [000/29931] mean_loss : 0.00
[85/100] [1000/29931] mean_loss : 0.02
[85/100] [2000/29931] mean_loss : 0.03
[85/100] [3000/29931] mean_loss : 0.02
[85/100] [4000/29931] mean_loss : 0.02
[85/100] [5000/29931] mean_loss : 0.03
[85/100] [6000/29931] mean_loss : 0.03
[85/100] [7000/29931] mean_loss : 0.03
[85/100] [8000/29931] mean_loss : 0.03
[85/100] [9000/29931] mean_loss : 0.04
[85/100] [10000/29931] mean_loss : 0.05
[85/100] [11000/29931] mean_loss : 0.05
[85/100] [12000/29931] mean_loss : 0.05
[85/100] [13000/29931] mean_loss : 0.05
[85/100] [14000/29931] mean_loss : 0.05
[85/100] [15000/29931] mean_loss : 0.

[90/100] [26000/29931] mean_loss : 0.05
[90/100] [27000/29931] mean_loss : 0.05
[90/100] [28000/29931] mean_loss : 0.05
[90/100] [29000/29931] mean_loss : 0.05
Precisions: 0.12957317073170818
Recall: 0.5185031939605111
F1: 0.2073339085830973
best result: 0.2166719467907358
2018-10-11 12:21:04
[91/100] [000/29931] mean_loss : 0.00
[91/100] [1000/29931] mean_loss : 0.02
[91/100] [2000/29931] mean_loss : 0.03
[91/100] [3000/29931] mean_loss : 0.02
[91/100] [4000/29931] mean_loss : 0.03
[91/100] [5000/29931] mean_loss : 0.03
[91/100] [6000/29931] mean_loss : 0.03
[91/100] [7000/29931] mean_loss : 0.03
[91/100] [8000/29931] mean_loss : 0.03
[91/100] [9000/29931] mean_loss : 0.04
[91/100] [10000/29931] mean_loss : 0.05
[91/100] [11000/29931] mean_loss : 0.05
[91/100] [12000/29931] mean_loss : 0.05
[91/100] [13000/29931] mean_loss : 0.05
[91/100] [14000/29931] mean_loss : 0.05
[91/100] [15000/29931] mean_loss : 0.05
[91/100] [16000/29931] mean_loss : 0.04
[91/100] [17000/29931] mean_loss : 0.

[96/100] [28000/29931] mean_loss : 0.05
[96/100] [29000/29931] mean_loss : 0.05
Precisions: 0.1292682926829277
Recall: 0.5157084785133567
F1: 0.20671986191342687
best result: 0.2166719467907358
2018-10-11 12:33:07
[97/100] [000/29931] mean_loss : 0.00
[97/100] [1000/29931] mean_loss : 0.02
[97/100] [2000/29931] mean_loss : 0.03
[97/100] [3000/29931] mean_loss : 0.03
[97/100] [4000/29931] mean_loss : 0.03
[97/100] [5000/29931] mean_loss : 0.03
[97/100] [6000/29931] mean_loss : 0.03
[97/100] [7000/29931] mean_loss : 0.03
[97/100] [8000/29931] mean_loss : 0.03
[97/100] [9000/29931] mean_loss : 0.03
[97/100] [10000/29931] mean_loss : 0.04
[97/100] [11000/29931] mean_loss : 0.04
[97/100] [12000/29931] mean_loss : 0.05
[97/100] [13000/29931] mean_loss : 0.05
[97/100] [14000/29931] mean_loss : 0.05
[97/100] [15000/29931] mean_loss : 0.05
[97/100] [16000/29931] mean_loss : 0.05
[97/100] [17000/29931] mean_loss : 0.05
[97/100] [18000/29931] mean_loss : 0.05
[97/100] [19000/29931] mean_loss : 0.

In [11]:
import NeuralPITF
from imp import reload
reload(NeuralPITF)
from NeuralPITF import AttentionPITF, SinglePITF_Loss, DataSet, RNNAttentionPITF, TagAttentionPITF
train(movielens, movielens_test, m, gamma)

/home/cbd109/applications/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Use the BPR for Optimization
2018-10-10 19:22:46
[00/100] [000/29931] mean_loss : 12.32
[00/100] [1000/29931] mean_loss : 0.27
[00/100] [2000/29931] mean_loss : 0.19
[00/100] [3000/29931] mean_loss : 0.13
[00/100] [4000/29931] mean_loss : 0.10
[00/100] [5000/29931] mean_loss : 0.09
[00/100] [6000/29931] mean_loss : 0.08
[00/100] [7000/29931] mean_loss : 0.07
[00/100] [8000/29931] mean_loss : 0.07
[00/100] [9000/29931] mean_loss : 0.06
[00/100] [10000/29931] mean_loss : 0.05
[00/100] [11000/29931] mean_loss : 0.05
[00/100] [12000/29931] mean_loss : 0.04
[00/100] [13000/29931] mean_loss : 0.04
[00/100] [14000/29931] mean_loss : 0.04
[00/100] [15000/29931] mean_loss : 0.04
[00/100] [16000/29931] mean_loss : 0.05
[00/100] [17000/29931] mean_loss : 0.05
[00/100] [18000/29931] mean_loss : 0.04
[00/100] [19000/29931] mean_loss : 0.04
[00/100] [20000/29931] mean_loss : 0.05
[00/100] [21000/29931] mean_loss : 0.04
[00/100] [22000/29931] mean_loss : 0.05
[00/100] [23000/29931] mean_loss : 0.06
[

[06/100] [000/29931] mean_loss : 0.00
[06/100] [1000/29931] mean_loss : 0.02
[06/100] [2000/29931] mean_loss : 0.02
[06/100] [3000/29931] mean_loss : 0.03
[06/100] [4000/29931] mean_loss : 0.03
[06/100] [5000/29931] mean_loss : 0.03
[06/100] [6000/29931] mean_loss : 0.03
[06/100] [7000/29931] mean_loss : 0.03
[06/100] [8000/29931] mean_loss : 0.03
[06/100] [9000/29931] mean_loss : 0.03
[06/100] [10000/29931] mean_loss : 0.04
[06/100] [11000/29931] mean_loss : 0.04
[06/100] [12000/29931] mean_loss : 0.04
[06/100] [13000/29931] mean_loss : 0.04
[06/100] [14000/29931] mean_loss : 0.04
[06/100] [15000/29931] mean_loss : 0.04
[06/100] [16000/29931] mean_loss : 0.04
[06/100] [17000/29931] mean_loss : 0.04
[06/100] [18000/29931] mean_loss : 0.05
[06/100] [19000/29931] mean_loss : 0.04
[06/100] [20000/29931] mean_loss : 0.04
[06/100] [21000/29931] mean_loss : 0.04
[06/100] [22000/29931] mean_loss : 0.04
[06/100] [23000/29931] mean_loss : 0.04
[06/100] [24000/29931] mean_loss : 0.05
[06/100] [2

[12/100] [2000/29931] mean_loss : 0.03
[12/100] [3000/29931] mean_loss : 0.03
[12/100] [4000/29931] mean_loss : 0.03
[12/100] [5000/29931] mean_loss : 0.03
[12/100] [6000/29931] mean_loss : 0.03
[12/100] [7000/29931] mean_loss : 0.03
[12/100] [8000/29931] mean_loss : 0.04
[12/100] [9000/29931] mean_loss : 0.04
[12/100] [10000/29931] mean_loss : 0.05
[12/100] [11000/29931] mean_loss : 0.06
[12/100] [12000/29931] mean_loss : 0.06
[12/100] [13000/29931] mean_loss : 0.06
[12/100] [14000/29931] mean_loss : 0.06
[12/100] [15000/29931] mean_loss : 0.06
[12/100] [16000/29931] mean_loss : 0.06
[12/100] [17000/29931] mean_loss : 0.06
[12/100] [18000/29931] mean_loss : 0.06
[12/100] [19000/29931] mean_loss : 0.06
[12/100] [20000/29931] mean_loss : 0.06
[12/100] [21000/29931] mean_loss : 0.06
[12/100] [22000/29931] mean_loss : 0.06
[12/100] [23000/29931] mean_loss : 0.07
[12/100] [24000/29931] mean_loss : 0.07
[12/100] [25000/29931] mean_loss : 0.06
[12/100] [26000/29931] mean_loss : 0.07
[12/100]

[18/100] [3000/29931] mean_loss : 0.04
[18/100] [4000/29931] mean_loss : 0.04
[18/100] [5000/29931] mean_loss : 0.04
[18/100] [6000/29931] mean_loss : 0.04
[18/100] [7000/29931] mean_loss : 0.05
[18/100] [8000/29931] mean_loss : 0.04
[18/100] [9000/29931] mean_loss : 0.06
[18/100] [10000/29931] mean_loss : 0.07
[18/100] [11000/29931] mean_loss : 0.07
[18/100] [12000/29931] mean_loss : 0.07
[18/100] [13000/29931] mean_loss : 0.07
[18/100] [14000/29931] mean_loss : 0.08
[18/100] [15000/29931] mean_loss : 0.08
[18/100] [16000/29931] mean_loss : 0.08
[18/100] [17000/29931] mean_loss : 0.08
[18/100] [18000/29931] mean_loss : 0.08
[18/100] [19000/29931] mean_loss : 0.08
[18/100] [20000/29931] mean_loss : 0.08
[18/100] [21000/29931] mean_loss : 0.08
[18/100] [22000/29931] mean_loss : 0.08
[18/100] [23000/29931] mean_loss : 0.08
[18/100] [24000/29931] mean_loss : 0.08
[18/100] [25000/29931] mean_loss : 0.09
[18/100] [26000/29931] mean_loss : 0.09
[18/100] [27000/29931] mean_loss : 0.09
[18/100

...

[72/100] [12000/29931] mean_loss : 0.08
[72/100] [13000/29931] mean_loss : 0.09
[72/100] [14000/29931] mean_loss : 0.10
[72/100] [15000/29931] mean_loss : 0.09
[72/100] [16000/29931] mean_loss : 0.09
[72/100] [17000/29931] mean_loss : 0.10
[72/100] [18000/29931] mean_loss : 0.11
[72/100] [19000/29931] mean_loss : 0.10
[72/100] [20000/29931] mean_loss : 0.10
[72/100] [21000/29931] mean_loss : 0.09
[72/100] [22000/29931] mean_loss : 0.12
[72/100] [23000/29931] mean_loss : 0.12
[72/100] [24000/29931] mean_loss : 0.14
[72/100] [25000/29931] mean_loss : 0.15
[72/100] [26000/29931] mean_loss : 0.16
[72/100] [27000/29931] mean_loss : 0.17
[72/100] [28000/29931] mean_loss : 0.19
[72/100] [29000/29931] mean_loss : 0.18
Precisions: 0.10670731707317127
Recall: 0.4400442799070848
F1: 0.17176335565043174
best result: 0.21480584026011826
2018-10-10 22:11:51
[73/100] [000/29931] mean_loss : 0.01
[73/100] [1000/29931] mean_loss : 0.05
[73/100] [2000/29931] mean_loss : 0.05
[73/100] [3000/29931] mean_l

[78/100] [13000/29931] mean_loss : 0.08
[78/100] [14000/29931] mean_loss : 0.10
[78/100] [15000/29931] mean_loss : 0.10
[78/100] [16000/29931] mean_loss : 0.08
[78/100] [17000/29931] mean_loss : 0.08
[78/100] [18000/29931] mean_loss : 0.09
[78/100] [19000/29931] mean_loss : 0.10
[78/100] [20000/29931] mean_loss : 0.10
[78/100] [21000/29931] mean_loss : 0.09
[78/100] [22000/29931] mean_loss : 0.11
[78/100] [23000/29931] mean_loss : 0.16
[78/100] [24000/29931] mean_loss : 0.13
[78/100] [25000/29931] mean_loss : 0.12
[78/100] [26000/29931] mean_loss : 0.16
[78/100] [27000/29931] mean_loss : 0.16
[78/100] [28000/29931] mean_loss : 0.18
[78/100] [29000/29931] mean_loss : 0.19
Precisions: 0.10365853658536639
Recall: 0.42657883275261327
F1: 0.16678770716080643
best result: 0.21480584026011826
2018-10-10 22:24:03
[79/100] [000/29931] mean_loss : 0.03
[79/100] [1000/29931] mean_loss : 0.03
[79/100] [2000/29931] mean_loss : 0.05
[79/100] [3000/29931] mean_loss : 0.05
[79/100] [4000/29931] mean_l

[84/100] [14000/29931] mean_loss : 0.09
[84/100] [15000/29931] mean_loss : 0.08
[84/100] [16000/29931] mean_loss : 0.09
[84/100] [17000/29931] mean_loss : 0.09
[84/100] [18000/29931] mean_loss : 0.09
[84/100] [19000/29931] mean_loss : 0.10
[84/100] [20000/29931] mean_loss : 0.10
[84/100] [21000/29931] mean_loss : 0.10
[84/100] [22000/29931] mean_loss : 0.10
[84/100] [23000/29931] mean_loss : 0.12
[84/100] [24000/29931] mean_loss : 0.13
[84/100] [25000/29931] mean_loss : 0.15
[84/100] [26000/29931] mean_loss : 0.14
[84/100] [27000/29931] mean_loss : 0.17
[84/100] [28000/29931] mean_loss : 0.18
[84/100] [29000/29931] mean_loss : 0.18
Precisions: 0.10457317073170787
Recall: 0.43077090592334494
F1: 0.1682920628274752
best result: 0.21480584026011826
2018-10-10 22:36:22
[85/100] [000/29931] mean_loss : 0.02
[85/100] [1000/29931] mean_loss : 0.03
[85/100] [2000/29931] mean_loss : 0.04
[85/100] [3000/29931] mean_loss : 0.04
[85/100] [4000/29931] mean_loss : 0.05
[85/100] [5000/29931] mean_los

[90/100] [15000/29931] mean_loss : 0.10
[90/100] [16000/29931] mean_loss : 0.10
[90/100] [17000/29931] mean_loss : 0.10
[90/100] [18000/29931] mean_loss : 0.09
[90/100] [19000/29931] mean_loss : 0.11
[90/100] [20000/29931] mean_loss : 0.12
[90/100] [21000/29931] mean_loss : 0.10
[90/100] [22000/29931] mean_loss : 0.11
[90/100] [23000/29931] mean_loss : 0.12
[90/100] [24000/29931] mean_loss : 0.12
[90/100] [25000/29931] mean_loss : 0.15
[90/100] [26000/29931] mean_loss : 0.15
[90/100] [27000/29931] mean_loss : 0.17
[90/100] [28000/29931] mean_loss : 0.17
[90/100] [29000/29931] mean_loss : 0.18
Precisions: 0.09969512195122
Recall: 0.41243829849012775
F1: 0.16057568135230113
best result: 0.21480584026011826
2018-10-10 22:48:38
[91/100] [000/29931] mean_loss : 0.00
[91/100] [1000/29931] mean_loss : 0.03
[91/100] [2000/29931] mean_loss : 0.05
[91/100] [3000/29931] mean_loss : 0.06
[91/100] [4000/29931] mean_loss : 0.04
[91/100] [5000/29931] mean_loss : 0.05
[91/100] [6000/29931] mean_loss :

[96/100] [16000/29931] mean_loss : 0.11
[96/100] [17000/29931] mean_loss : 0.12
[96/100] [18000/29931] mean_loss : 0.10
[96/100] [19000/29931] mean_loss : 0.11
[96/100] [20000/29931] mean_loss : 0.10
[96/100] [21000/29931] mean_loss : 0.09
[96/100] [22000/29931] mean_loss : 0.10
[96/100] [23000/29931] mean_loss : 0.12
[96/100] [24000/29931] mean_loss : 0.12
[96/100] [25000/29931] mean_loss : 0.12
[96/100] [26000/29931] mean_loss : 0.15
[96/100] [27000/29931] mean_loss : 0.16
[96/100] [28000/29931] mean_loss : 0.16
[96/100] [29000/29931] mean_loss : 0.18
Precisions: 0.10182926829268349
Recall: 0.4275043554006969
F1: 0.16448022099425413
best result: 0.21480584026011826
2018-10-10 23:00:57
[97/100] [000/29931] mean_loss : 0.02
[97/100] [1000/29931] mean_loss : 0.03
[97/100] [2000/29931] mean_loss : 0.05
[97/100] [3000/29931] mean_loss : 0.06
[97/100] [4000/29931] mean_loss : 0.07
[97/100] [5000/29931] mean_loss : 0.05
[97/100] [6000/29931] mean_loss : 0.05
[97/100] [7000/29931] mean_loss 